In [3]:
import sys
sys.path.insert(0, "..")

from create_daily_csv import readsnow_in_shp_db, xarray2df, readnetcdf_in_shp_db
from create_daily_csv import spatial_stats_daily_input
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plot
import xarray as xr
import datetime
import psycopg2
import matplotlib.pyplot as plt

from joblib import load
from base_f import create_it_matrix
from db_insert import insert



In [4]:

LIST=['ADO_DSC_CH03_0075',
     'ADO_DSC_AT31_0254',
     'ADO_DSC_ITC1_0072',
     'ADO_DSC_ITC1_0020',
     'ADO_DSC_CH07_0147',
     'ADO_DSC_AT31_0206',
     'ADO_DSC_ITH1_0012',
     'ADO_DSC_AT12_0280',
     'ADO_DSC_CH07_0100',
     'ADO_DSC_CH05_0201',
     'ADO_DSC_SI03_0148',
     'ADO_DSC_ITC1_0037',
     'ADO_DSC_FRK2_0042',
     'ADO_DSC_CH04_0011',
     'ADO_DSC_ITH2_0035',
     'ADO_DSC_SI03_0033',
     'ADO_DSC_FRK2_0041',
     'ADO_DSC_ITH5_0006']


era5_fileName_t=  'Z:\ADO\ZAMG\downscaled_archive\\2m_temperature-19790101_20201231-eusalp-era5_qm.nc'
era5_fileName_e = 'Z:\ADO\ZAMG\downscaled_archive\\potential_evapotranspiration-19790101_20201231-eusalp-qm_era5.nc'
era5_fileName_p = 'Z:\ADO\ZAMG\downscaled_archive\\total_precipitation-19790101_20201231-eusalp-qm_era5.nc'
era5_foldName_s = 'Z:\ADO\ZAMG\SNOWGRID\\'
# mask file needed to exclude pixels on the glaciers.
mask_file=r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\era5\SNOWGRID\\snowgrid_masks.nc'


import sys, importlib
importlib.reload(sys.modules['create_daily_csv'])

In [13]:
    conn = psycopg2.connect(host="10.8.244.31",
                       database="climate_data",
                       user="ado_admin",
                       password="oda347hydro",
                       port=5432)
                       
    cur = conn.cursor()
    
    # get the metadata
    query = f"""
            SELECT "id_station", MAX("date") FROM "ML_discharge"."mod_disc" 
            GROUP BY "id_station"    
            """
    df = pd.read_sql_query(query,conn)
    
    # close the connection when finished
    cur.close()
    conn.close()

    df.index=df.id_station

In [33]:
for STAT_CODE in LIST:
    STAT_CODE
    
    t2m = readnetcdf_in_shp_db(era5_fileName_t,STAT_CODE ,plot=False,res=5500)['t2m']
    
    #select dates
    last_mod_date=np.datetime64(df.loc[STAT_CODE][1])
    last_data_date=np.datetime64(np.array(t2m.time[-1]),'D')
    
    if  (last_mod_date < last_data_date):


        t2m = t2m.sel(time=slice(last_mod_date - np.timedelta64(365,'D'),last_data_date))
        t2m = xarray2df(t2m.resample(time='1d').sum(skipna=False), 'T','t2m')

        ### ERA5 total precipitation

        #CLIP TO THE SHAPEFILE
        tp = readnetcdf_in_shp_db(era5_fileName_p,STAT_CODE ,plot=False,res=5500)['tp']
        tp = tp.sel(time=slice(last_mod_date - np.timedelta64(365,'D'),last_data_date))
        tp = xarray2df(tp.resample(time='1d').sum(skipna=False), 'P','tp')


        ### ERA5 evapotranspiration

        #CLIP TO THE SHAPEFILE
        pet = readnetcdf_in_shp_db(era5_fileName_e,STAT_CODE ,plot=False,res=5500)['pet']
        pet = pet.sel(time=slice(last_mod_date - np.timedelta64(365,'D'),last_data_date))
        pet = xarray2df(pet.resample(time='1d').sum(skipna=False), 'E','pet')


        ### ERA5 SNOW VARIABLES

        #CLIP TO THE SHAPEFILE
        s = readsnow_in_shp_db(era5_foldName_s,mask_file,STAT_CODE ,plot=False,res=5500)['swe_tot']
        s = s.sel(time=slice(last_mod_date - np.timedelta64(365,'D'),last_data_date))
        s = xarray2df(s.resample(time='1d').sum(skipna=False), 'S','swe_tot')

        #CONCATENATE THE VARIABLES
        daily_input = pd.concat([t2m, s, tp, pet], axis=1, join='inner')
        daily_input_stat = spatial_stats_daily_input(daily_input)
        daily_input_stat['Q']=0
        in_matrix=create_it_matrix(daily_input_stat,36,10)
        
        #select the the input data interesting
        in_matrix = in_matrix[str((last_mod_date).astype('datetime64[D]')):str(last_data_date.astype('datetime64[D]'))]


        #load the model
        fld=r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\Documents\conda\Runoff_prediction\model_train\models\\'
        model=load(fld+STAT_CODE+'.joblib')

        #predict the discharge and add ancillary information

        data=model.predict(in_matrix.drop(columns='Q'))

        discharge = pd.DataFrame(data=data ,index=in_matrix.index ,columns=['prediction'])

        discharge['meas_disch_presence'] = False

        plt.figure()
        discharge.prediction.plot.line(color='red',figsize=(10,10))



        conn = psycopg2.connect(host="10.8.244.31",
                           database="climate_data",
                           user="ado_admin",
                           password="oda347hydro",
                           port=5432)

        cur = conn.cursor()

        # get the metadata
        query = f"""
                SELECT "discharge", "date","meas_disch_presence" FROM "ML_discharge"."mod_disc" 
                WHERE "id_station" = '{STAT_CODE}'
                AND "date" > '{(last_mod_date-np.timedelta64(365,'D'))}'
                """
        old = pd.read_sql_query(query,conn)

        # close the connection when finished
        cur.close()
        conn.close()

        for i in range(old.shape[0]):
            old.loc[i,'ind'] = np.datetime64(old.iloc[i].date)

        old.index=old.ind

        old.discharge.plot.line(color='blue')
        
        insert(STAT_CODE , discharge.iloc[1:])

with open('log.csv','a') as fd:
    fd.write(str(last_data_date)+'\n')

# test

In [14]:
df

,id_station,max
id_station,,
ADO_DSC_ITC1_0020,ADO_DSC_ITC1_0020,2020-12-31
ADO_DSC_ITC1_0072,ADO_DSC_ITC1_0072,2020-12-31
ADO_DSC_AT31_0254,ADO_DSC_AT31_0254,2020-12-31
ADO_DSC_CH07_0006,ADO_DSC_CH07_0006,2020-12-01
ADO_DSC_CH03_0075,ADO_DSC_CH03_0075,2020-12-31
ADO_DSC_AT31_0206,ADO_DSC_AT31_0206,2020-12-31
ADO_DSC_CH07_0147,ADO_DSC_CH07_0147,2020-12-31
ADO_DSC_AT12_0280,ADO_DSC_AT12_0280,2020-12-31
ADO_DSC_ITH1_0012,ADO_DSC_ITH1_0012,2020-12-31
